In [1]:
import pandas as pd
import numpy as np
import json

In [8]:
meta = pd.read_csv('DataSet/movies_metadata/movies_metadata.csv', low_memory=False)
meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


## movie_metadata 전처리 과정

genres의 id와 movieId는 영화의 고유번호로 사용자 평가 데이터와 연결할 때 인덱스 값으로 쓸 예정

In [9]:
meta = meta[['id', 'original_title', 'original_language', 'genres']]
meta = meta.rename(columns={'id':'movieId'})  #id를 movieId로 변경(식별 편하게)
meta = meta[meta['original_language'] == 'en']   #영어로 된 영어들이 많아서 영어로 된 영화만 추림
meta.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


## 사용자 평가 데이터 전처리 과정(small데이터)

In [10]:
ratings = pd.read_csv('DataSet/ratings_small/ratings_small.csv', low_memory=False)
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [11]:
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [12]:
ratings.describe()

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


rating값은 0,5~5.0이라는 것을 알 수 있습니다.

### 데이터 가공

In [13]:
meta.movieId = pd.to_numeric(meta.movieId, errors='coerce')  #to_numeric() : 문자열을 숫자 타입으로 변환, 연산을 위해서
ratings.movieId = pd.to_numeric(ratings.movieId, errors='coerce')
#to_numeric()의 errors 3가지 : ignore -> 숫자로 변경 불가능 시 원본데이터 반환
#coerce -> 숫자로 변경 불가능 시 기존 데이터 지우고 NaN으로 설정하고 반환
#raise -> 숫자로 변경 불가능 시 에러를 일으키고 코드 중단

genres는 json형식의 객체가 담긴 array이다. 이를 배열로 변경하기 위해 함수를 하나 선언한다. 배열로 바꾸는 이유는 인덱스로 값을 다루기 위해서이다. 

In [15]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))  #single quote를 double quote로 변경
    
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])
        
    return genres_list

meta['genres'] = meta['genres'].apply(parse_genres)  #apply()를 통해 각 행에 해당 함수 적용
meta.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[Animation, Comedy, Family]"
1,8844,Jumanji,en,"[Adventure, Fantasy, Family]"
2,15602,Grumpier Old Men,en,"[Romance, Comedy]"
3,31357,Waiting to Exhale,en,"[Comedy, Drama, Romance]"
4,11862,Father of the Bride Part II,en,[Comedy]


## Merge Meta and Ratings

In [16]:
data = pd.merge(ratings, meta, on='movieId', how='inner')   #movieId를 기준으로, inner방식으로 병합
data.head()

,userId,movieId,rating,original_title,original_language,genres
0,1,1371,2.5,Rocky III,en,[Drama]
1,4,1371,4.0,Rocky III,en,[Drama]
2,7,1371,3.0,Rocky III,en,[Drama]
3,19,1371,4.0,Rocky III,en,[Drama]
4,21,1371,3.0,Rocky III,en,[Drama]


## pivot table

In [17]:
matrix = data.pivot_table(index='userId', columns='original_title', values='rating')
# values:분석할 데이터 프레임에서 분석할 열
#https://datascienceschool.net/01%20python/04.07%20%ED%94%BC%EB%B4%87%ED%85%8C%EC%9D%B4%EB%B8%94%EA%B3%BC%20%EA%B7%B8%EB%A3%B9%EB%B6%84%EC%84%9D.html
matrix.head(20)

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Pearson Correlation

상관관계는 두 변수 사이의 밀접성(선형관계)강도와 방향을 요약하는 수치
두 변수간의 상관관계가 0이면 서로 상관이x이게 아니라 선형의 상관관계가 아니라는 뜻
독립과 종속의 인과관계를 파악하려면 상관분석이 아닌 '회귀분석'을 해야한다.
피어슨 상관분석은 두 변수 모두 등간 또는 비율척도(즉, 양의 값을 가질 때)일 때 사용하는 분석이다.
피어슨 상관계수 r은 다음과 같다.
r = "X와Y가 함께 변하는 정도"/"X와 Y가 각각 변하는 정도
X와 Y가 완전 동일시 +1, 반대방향으로 완전히 동일하면 -1의 값을 갖는다.결정계수는 r^2로 계산하며 이것은 X로부터 Y를 예측할 수 있는 정도를 의미한다.

In [22]:
GENRE_WEIGHT = 0.1

def pearsonR(s1,s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c**2) * np.sum(s2_c **2))
#분자가 클수록 상관관계 값도 커진다.

#input_movie : 찾고싶은 영화의 타이틀, matrix : Pivot Table, n : 몇 개의 영화를 추천 받을 것인지,
#similar_genre : 비슷한 영화의 가중치를 둘 것인지.
def recommend(input_movie, matrix, n, similar_genre=True) :   #검색 엔진 함수로 구현
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]
    
    result = []
    for title in matrix.columns :
        if title == input_movie :  #똑같은 영화가 추천되면 안되므로 스킵
            continue
        
        cor = pearsonR(matrix[input_movie], matrix[title])
        
        #temp_genres와 같은 선언에 대해서
        #https://azanewta.tistory.com/34
        if similar_genre and len(input_genres) > 0:
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]
            same_count = np.sum(np.isin(input_genres, temp_genres))
            #np.isin() : 배열을 비교하여 똑같은 요소가 있으면 True반환
            cor += (GENRE_WEIGHT * same_count)  #같은 장르가 많을수록 가중치를 준다.
            
        if np.isnan(cor):
            continue
        else :
            result.append((title, '{:.2f}'.format(cor), temp_genres))
            
    result.sort(key = lambda r:r[1], reverse=True)  #레이팅 높은 순서대로 정렬한다 내림차순으로
    
    return result[:n]

## Prediction

In [25]:
recommend_result = recommend('The Dark Knight', matrix, 10, similar_genre=True)

pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])

C:\Users\원우형\AppData\Local\Temp\ipykernel_2928\532188032.py:6: RuntimeWarning: invalid value encountered in double_scalars
  return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c**2) * np.sum(s2_c **2))


,Title,Correlation,Genre
0,Prom Night,0.87,"[Horror, Mystery, Thriller]"
1,Wild Wild West,0.87,"[Action, Adventure, Comedy, Science Fiction, W..."
2,Blue Thunder,0.73,"[Science Fiction, Action, Thriller, Crime, Drama]"
3,Topaz,0.68,"[Action, Drama, Mystery, Thriller]"
4,Yamakasi - Les samouraïs des temps modernes,0.68,"[Action, Crime, Drama]"
5,Best Seller,0.67,"[Action, Crime, Drama, Thriller]"
6,Midnight in the Garden of Good and Evil,0.67,"[Crime, Drama, Mystery, Thriller]"
7,Big Bad Mama,0.64,"[Action, Comedy, Crime, Drama]"
8,The Enforcer,0.63,"[Action, Crime, Thriller]"
9,The River Wild,0.63,"[Action, Adventure, Crime, Thriller]"


In [2]:
keyword_data = pd.read_csv('DataSet/keywords.csv', low_memory=False)
keyword_data.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [7]:
keywords = keyword_data['keywords']
keywords.head()

0    [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1    [{'id': 10090, 'name': 'board game'}, {'id': 1...
2    [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3    [{'id': 818, 'name': 'based on novel'}, {'id':...
4    [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
Name: keywords, dtype: object

In [10]:
todic = keyword_data['keywords'].to_dict()
todic

{0: "[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]",
 1: '[{\'id\': 10090, \'name\': \'board game\'}, {\'id\': 10941, \'name\': \'disappearance\'}, {\'id\': 15101, \'name\': "based on children\'s book"}, {\'id\': 33467, \'name\': \'new home\'}, {\'id\': 158086, \'name\': \'recluse\'}, {\'id\': 158091, \'name\': \'giant insect\'}]',
 2: "[{'id': 1495, 'name': 'fishing'}, {'id': 12392, 'name': 'best friend'}, {'id': 179431, 'name': 'duringcreditsstinger'}, {'id': 208510, 'name': 'old men'}]",
 3: "[{'id': 818, 'name': 'based on novel'}, {'id': 10131, 'name': 'interracial relationship'}, {'id': 14768, 'name': 'single mother'}, {'id': 15160, 'name': 'divorce'}, {'id': 33455, 'name': 'chick flick'}]",
 4: "[{'id': 1009, 'name'

In [21]:
print(keywords.str.replace('\"\'\"','\'\''))
test = keywords.str.replace('\"*\'*\"','\'**\'',regex=False)
condition = test.isin(['\'*\''])
print(condition)

0        [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1        [{'id': 10090, 'name': 'board game'}, {'id': 1...
2        [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3        [{'id': 818, 'name': 'based on novel'}, {'id':...
4        [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
                               ...                        
46414               [{'id': 10703, 'name': 'tragic love'}]
46415    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
46416                                                   []
46417                                                   []
46418                                                   []
Name: keywords, Length: 46419, dtype: object
0        False
1        False
2        False
3        False
4        False
         ...  
46414    False
46415    False
46416    False
46417    False
46418    False
Name: keywords, Length: 46419, dtype: bool


In [46]:
import re

# p = re.compile(".*(?!').*$")
# m = p.search("brandon's jeourny")
# print(m)
# print(m.group())
string = "brandon's jeourny"
new_str = re.sub(r"^.+['].+$","^.+.+$",string)
print(new_str)

^.+.+$


In [58]:
keyword_data['keywords'] = keyword_data['keywords'].str.extract(r'(^.+[\'].+$)')
keyword_data
len(keyword_data.keywords.str.contains(r'(.*[\'].*)'))

C:\Users\원우형\AppData\Local\Temp\ipykernel_29456\1914311618.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  len(keyword_data.keywords.str.contains(r'(.*[\'].*)'))


46419